In [6]:
import os
import re
import functools
from collections import OrderedDict
import doctest
from llparser import *
os.chdir('../form/sources')
os.getcwd()

'/home/feelus/tmp/form/sources'

In [7]:
os.listdir()

['grccparam.h',
 'Makefile.am',
 'sort.c',
 'polygcd.h',
 'compiler.c',
 'names.c',
 'if.c',
 'transform.c',
 'evaluate.c',
 'sch.c',
 'comtool.c',
 'lus.c',
 'gentopo.cc',
 'poly.h',
 'proces.c',
 'token.c',
 'normal.c',
 'mallocprotect.h',
 'declare.h',
 'fsizes.h',
 'symmetr.c',
 'inivar.h',
 'parallel.c',
 'tools.c',
 'minos.h',
 'gentopo.h',
 'wildcard.c',
 'startup.c',
 'pattern.c',
 'minos.c',
 'pre.c',
 'function.c',
 'reshuf.c',
 'parallel.h',
 'polyfact.h',
 'grcc.cc',
 'message.c',
 'diagrams.c',
 'setfile.c',
 'float.c',
 'comtool.h',
 'diawrap.cc',
 'threads.c',
 'notation.c',
 'smart.c',
 'mpidbg.h',
 'grcc.h',
 'spectator.c',
 'ratio.c',
 'topowrap.cc',
 'mytime.cc',
 'polygcd.cc',
 'ftypes.h',
 'findpat.c',
 'mpi.c',
 'dollar.c',
 'opera.c',
 'bugtool.c',
 'unixfile.c',
 'tables.c',
 'unix.h',
 'vector.h',
 'polywrap.cc',
 'form3.h',
 'dict.c',
 'structs.h',
 'model.c',
 'extcmd.c',
 'module.c',
 'execute.c',
 'argument.c',
 'optimize.cc',
 'factor.c',
 'compcomm.c',
 '

In [8]:
file1 = open('form3.h', 'r')
Lines = file1.readlines()
Lines

['/** @file form3.h\n',
 ' *\n',
 ' *  Contains critical defines for the compilation process\n',
 ' *\tAlso contains the inclusion of all necessary header files.\n',
 ' *\tThere are also some system dependencies concerning file functions.\n',
 ' */\n',
 '\n',
 '/* #[ License : */\n',
 '/*\n',
 ' *   Copyright (C) 1984-2023 J.A.M. Vermaseren\n',
 ' *   When using this file you are requested to refer to the publication\n',
 ' *   J.A.M.Vermaseren "New features of FORM" math-ph/0010025\n',
 ' *   This is considered a matter of courtesy as the development was paid\n',
 ' *   for by FOM the Dutch physics granting agency and we would like to\n',
 ' *   be able to track its scientific use to convince FOM of its value\n',
 ' *   for the community.\n',
 ' *\n',
 ' *   This file is part of FORM.\n',
 ' *\n',
 ' *   FORM is free software: you can redistribute it and/or modify it under the\n',
 ' *   terms of the GNU General Public License as published by the Free Software\n',
 ' *   Foundation, e

In [9]:
Lines[0]

'/** @file form3.h\n'

In [10]:
def strip_baks_comment(lines):
    for l in lines:
        r = ''
        p=0
        while (q:=l[p:].find('/*$')) !=-1:
            r+=l[p:q]
            p = l[q+3].index('$*/')
        q = l[p:].find('//$') if '//$'in l else -1
        r+=l[p:q]
        yield r

In [11]:
def end_merge(lines):
    r=''
    sep_positions = []
    for l in lines:
        if not l.endswith('\\'):
            yield (r+l,sep_positions)
            r=''
            sep_positions=[]
        else:
            r+=l.removesuffix('\\')
            sep_positions.append(sep_positions[-1]+len(l)-1 if len(sep_positions) else len(l)-1)
    if len(sep_positions):
        raise ValueError('aaa')

In [12]:
for x,sp in end_merge(strip_baks_comment(Lines)):
    if len(sp):
        print(x)

#define STATIC_ASSERT__3(X,L) 	typedef char static_assertion_failed_##L[(!!(X))*2-1]
#define PADDUMMY(type, size) 	UBYTE d_u_m_m_y[alignof(type) - ((size) & (alignof(type) - 1))]
#define PADPOSITION(ptr_,long_,int_,word_,byte_) 	PADDUMMY(off_t, 		+ sizeof(int *) * (ptr_) 		+ sizeof(LONG)  * (long_) 		+ sizeof(int)   * (int_) 		+ sizeof(WORD)  * (word_) 		+ sizeof(UBYTE) * (byte_) 	)
#define PADPOINTER(long_,int_,word_,byte_) 	PADDUMMY(int *, 		+ sizeof(LONG)  * (long_) 		+ sizeof(int)   * (int_) 		+ sizeof(WORD)  * (word_) 		+ sizeof(UBYTE) * (byte_) 	)
#define PADLONG(int_,word_,byte_) 	PADDUMMY(LONG, 		+ sizeof(int)   * (int_) 		+ sizeof(WORD)  * (word_) 		+ sizeof(UBYTE) * (byte_) 	)
#define PADINT(word_,byte_) 	PADDUMMY(int, 		+ sizeof(WORD)  * (word_) 		+ sizeof(UBYTE) * (byte_) 	)
#define PADWORD(byte_) 	PADDUMMY(WORD, 		+ sizeof(UBYTE) * (byte_) 	)
#define Utruncate(f) { 	if ( ftruncate(f->descriptor, 0) ) { 		MLOCK(ErrorMessageLock); 		MesPrint("Utruncate failed"); 		MUNLOCK(Er

In [13]:
def highlight_comments(lines):
    for l,sp in lines:
        pass

In [14]:
[x for x in reversed([1,2,3])]

[3, 2, 1]

In [20]:
USE_RE = False
def myprint(*args):
    print(*[repr(x) for x in args])
    return args[0]

punctuator        = char_in_set('[](){}<>.,+-&*/~!%=^|?:;#',errproc='punctuator')
digit             = char_in_set('0123456789'               ,errproc='digit')
hexadecimal_digit = char_in_set('0123456789abcdefABCDEF'   ,errproc='hexadecimal digit')
octal_digit       = char_in_set('01234567'                 ,errproc='octal digit')
nondigit          = char_in_set('_'+''.join(chr(i) for i in range(ord('a'),ord('z')+1))+\
                                    ''.join(chr(i) for i in range(ord('A'),ord('Z')+1)),errproc='nondigit')

@cacheread
def hexadecimal_prefix(s,pos):
    '''
    >>> hexadecimal_prefix('0x000',mkpos(0)) # doctest: 
    '0x'
    >>> hexadecimal_prefix('0000',mkpos(0)) # doctest: +ELLIPSIS
    parseError([(0, 'hexadecimal_prefix')...)
    '''
    if USE_RE:
        return read(s,pos, regexp(r'0[xX]',errproc='hexadecimal_prefix',proc=lambda x : x[0]))
    else:
        return read_oneof(s,pos,'0x','0X',errproc='hexadecimal_prefix')

@cacheread
def hex_quad(s,pos): # without universal character name
    '''
    >>> hex_quad('0134',mkpos(0)) # doctest: 
    308
    >>> hex_quad('013',mkpos(0)) # doctest:  +ELLIPSIS
    parseError([(0, 'hex_quad')...)
    '''
    if USE_RE:
        return read(s,pos, regexp(r'[0-9a-zA-Z]{4}',errproc='hex_quad',proc = lambda x : int(x[0],16)))
    else:
        return read_sequential(s,pos,a=hexadecimal_digit,b=hexadecimal_digit,c=hexadecimal_digit,d=hexadecimal_digit,
                         proc = lambda d: int(d.a+d.b+d.c+d.d,16), errproc = 'hex_quad' )

@cacheread
def universal_character_name(s,pos):
    # chr(92) == '\\'
    '''
    >>> universal_character_name(chr(92)+"u0130",mkpos(0)) # doctest: 
    'İ'
    >>> universal_character_name(chr(92)+"U000103a6",mkpos(0)) # doctest: 
    '𐎦'
    >>> universal_character_name('0134',mkpos(0)) # doctest: +ELLIPSIS, 
    parseError([(0, 'universal_character_name')...)
    '''
    if USE_RE:
        return read(s,pos, regexp(r'\\u[0-9a-zA-Z]{4}|\\U[0-9a-zA-Z]{8}',errproc='universal_character_name',
                                  proc=lambda x : chr(int(x[0][2:],16))))
    else:
        return read_oneof(s,pos,sequential(a='\\u',b=hex_quad, proc= lambda d: chr(d.b) , errproc= '\\uhhhh' ),
                sequential(a='\\U',b=hex_quad,c=hex_quad, proc= lambda d: chr(d.b*65536+d.c) , errproc= '\\Uhhhhhhhh' ),
                         errproc='universal_character_name')

@cacheread
def identifier(s,pos):
    '''
    >>> identifier('as0df',mkpos(0)) # doctest:
    'as0df'
    >>> identifier('0as0df',mkpos(0)) # doctest: +ELLIPSIS
    parseError([(0, 'identifier')...)
    '''
    if USE_RE:
        return read(s,pos, regexp(r'[a-zA-Z_][a-zA_Z0-9_]*',errproc='identifier')) # without universal character name
    else:
        return read_sequential(s,pos, f=oneof(nondigit,universal_character_name), 
                                      s=repeatedly(0,infinity,oneof(nondigit,universal_character_name,digit), proc=lambda l:''.join(l)) ,
                              proc=lambda d: d.f+d.s, errproc= 'identifier'
                              )

doctest.testmod()

TestResults(failed=0, attempted=9)

In [21]:
USE_RE = False

In [22]:
hexadecimal_prefix('0x000',mkpos(0))

'0x'

In [23]:
hexadecimal_prefix('0000',mkpos(0)) # doctest: +ELLIPSIS

parseError([(0, 'hexadecimal_prefix')], [parseError([(0, '0x')], None), parseError([(0, '0X')], None)])

In [24]:
hex_quad('0134',mkpos(0))

308

In [25]:
hex_quad('013',mkpos(0))

parseError([(0, 'hex_quad'), (3, 'hexadecimal digit')], None)

In [26]:
universal_character_name('\\u0130',mkpos(0))

'İ'

In [27]:
universal_character_name('\\U000103a6',mkpos(0))

'𐎦'

In [28]:
universal_character_name('0134',mkpos(0))

parseError([(0, 'universal_character_name')], [parseError([(0, '\\uhhhh'), (0, '\\u')], None), parseError([(0, '\\Uhhhhhhhh'), (0, '\\U')], None)])

In [29]:
identifier('as0d f',mkpos(0))

'as0d'

In [30]:
identifier('0as0df',mkpos(0))

parseError([(0, 'identifier'), (0, 'one of')], [parseError([(0, 'nondigit')], None), parseError([(0, 'universal_character_name')], [parseError([(0, '\\uhhhh'), (0, '\\u')], None), parseError([(0, '\\Uhhhhhhhh'), (0, '\\U')], None)])])

In [31]:
parseError([(0, 'one of')], [parseError([(0, '0x')], None), parseError([(0, '0X')], None)])

parseError([(0, 'one of')], [parseError([(0, '0x')], None), parseError([(0, '0X')], None)])

In [32]:
s=''
start = 65536
for i in range(start,start+1000):
    print(i,chr(i))


65536 𐀀
65537 𐀁
65538 𐀂
65539 𐀃
65540 𐀄
65541 𐀅
65542 𐀆
65543 𐀇
65544 𐀈
65545 𐀉
65546 𐀊
65547 𐀋
65548 𐀌
65549 𐀍
65550 𐀎
65551 𐀏
65552 𐀐
65553 𐀑
65554 𐀒
65555 𐀓
65556 𐀔
65557 𐀕
65558 𐀖
65559 𐀗
65560 𐀘
65561 𐀙
65562 𐀚
65563 𐀛
65564 𐀜
65565 𐀝
65566 𐀞
65567 𐀟
65568 𐀠
65569 𐀡
65570 𐀢
65571 𐀣
65572 𐀤
65573 𐀥
65574 𐀦
65575 𐀧
65576 𐀨
65577 𐀩
65578 𐀪
65579 𐀫
65580 𐀬
65581 𐀭
65582 𐀮
65583 𐀯
65584 𐀰
65585 𐀱
65586 𐀲
65587 𐀳
65588 𐀴
65589 𐀵
65590 𐀶
65591 𐀷
65592 𐀸
65593 𐀹
65594 𐀺
65595 𐀻
65596 𐀼
65597 𐀽
65598 𐀾
65599 𐀿
65600 𐁀
65601 𐁁
65602 𐁂
65603 𐁃
65604 𐁄
65605 𐁅
65606 𐁆
65607 𐁇
65608 𐁈
65609 𐁉
65610 𐁊
65611 𐁋
65612 𐁌
65613 𐁍
65614 𐁎
65615 𐁏
65616 𐁐
65617 𐁑
65618 𐁒
65619 𐁓
65620 𐁔
65621 𐁕
65622 𐁖
65623 𐁗
65624 𐁘
65625 𐁙
65626 𐁚
65627 𐁛
65628 𐁜
65629 𐁝
65630 𐁞
65631 𐁟
65632 𐁠
65633 𐁡
65634 𐁢
65635 𐁣
65636 𐁤
65637 𐁥
65638 𐁦
65639 𐁧
65640 𐁨
65641 𐁩
65642 𐁪
65643 𐁫
65644 𐁬
65645 𐁭
65646 𐁮
65647 𐁯
65648 𐁰
65649 𐁱
65650 𐁲
65651 𐁳
65652 𐁴
65653 𐁵
65654 𐁶
65655 𐁷
65656 𐁸
65657 𐁹
65658 𐁺
65659 𐁻
65660 𐁼


In [236]:
print(chr(66520))

𐏘


In [53]:
hex(66470)

'0x103a6'

In [104]:
def gen_fun(name):
    def default_name():
        return 1/0
    default_name.__name__ = name
    return default_name
foo = gen_fun('hello')
pass
pass
pass
foo()

ZeroDivisionError: division by zero